In [1]:
from builtins import Exception
import urllib3
from bs4 import BeautifulSoup
from requests import get
import mysql.connector
import re
import datetime

# Class Liputan6 Scraping

class dibawah merupakan proses pengambilan data dari website berita Liputan6.com yang dimana pemanggilan semua fungsi dibawah tersebut nantinya akan dipanggil pada fungsi [scrapingDataLiputan6]

In [16]:
class Liputan6:
     #deklarasi variable global
    url = ""
    mydb = ""
    keyword = ""
    urlDetailData = ""
    date = ""
    
    #fungsi dari inisialisasi data saat memanggil class pertama kali
    def __init__(self, urlBerita, pencarian,searchDate):
        self.url = urlBerita
        self.keyword = pencarian
        try:
            if(searchDate == ""):
                self.date = datetime.datetime.now()
            else:
                self.date = datetime.datetime.strptime(searchDate, '%d-%m-%Y')
        except Exception as e:
            print(e)
        self.mydb = mysql.connector.connect(host="localhost",user="root", password="",database="db-berita-mca")
    
    #fungsi menyimpan data ke database
    def simpanDatabase(self,data):
        mycursor = self.mydb.cursor()
        sql = "INSERT INTO tb_berita (media,judul_berita,label,penulis,release_date,url,content) VALUES (%s,%s,%s,%s,%s,%s,%s)"
        mycursor.execute(sql, data)
        self.mydb.commit()
        return
    
    #fungsi buat validasi data duplicate di database
    def validasiDataDatabase(self,dataBerita):
        mycursor = self.mydb.cursor()
        sql = "SELECT * FROM tb_berita WHERE url = %s"
        dataUrl = dataBerita
        url = (dataBerita,)
        mycursor.execute(sql, url)
        myresult = mycursor.fetchall()
        return myresult
        
    def getUrlDetail(self,page):
        self.urlDetailData = []
        for page in range(page): 
            try:
                if(self.keyword == ""):
                    urlGet = (self.url+"/news/indeks/"+self.date.strftime("%Y/%m/%d")+"?page={}".format(page+1))
                    soup = BeautifulSoup(get(urlGet).text, 'lxml')
                    getContent = soup.find('article','main')
                    groupContent = getContent.find_all('h4','articles--rows--item__title')
                    for x in groupContent:
                        news_url = x.find('a', href=True).get('href')
                        mergeUrl = (news_url+"?page=all")
                        self.urlDetailData.append(mergeUrl)
                else:
                    dateNow = self.date.strftime("%d/%m/%Y")
                    date = dateNow.replace('/','%2F')
                    urlGet = (self.url+"/search?order=latest&chammel_id=&from_date="+date+"&to_date="+date+"&type=all&q="+self.keyword)
                    soup = BeautifulSoup(get(urlGet).text, 'lxml')
                    getContent = soup.find('article','main')
                    groupContent = getContent.find_all('h4','articles--iridescent-list--text-item__title')
                    for x in groupContent:
                        news_url = x.find('a', href=True).get('href')
                        mergeUrl = (news_url+"?page=all")
                        self.urlDetailData.append(mergeUrl)
            except Exception as e:
                print("format tanggal anda masukan salah")
                break
                    
                    
    def scrapingLiputan6(self):
        self.getUrlDetail(5)
        for dataBerita in self.urlDetailData:
            myresult = self.validasiDataDatabase(dataBerita)
            if(len(myresult)==0):
                try:
                    media = "Liputan6.com"
                    web_data = get(dataBerita)
                    soup = BeautifulSoup(web_data.text, 'html.parser')
                    page = soup.find('div','inner-container-article')
                    title = page.find('h1','read-page--header--title entry-title').text
                    labelGroup = page.find_all('a','read-page--breadcrumb--item__title')
                    for x in labelGroup:
                        label = x.find('span').text
                    author = page.find('span','read-page--header--author__name fn').text
                    release_date = page.find('time','read-page--header--author__datetime updated').text      
                    groupContent = page.find('div','read-page--content')
                    texts = groupContent.find_all('p')
                    news_content = ' '.join([text.text for text in texts])           
                    data = (media,title,label,author,release_date,dataBerita,news_content)
                    self.simpanDatabase(data)
                except Exception as e:
                    continue 
            else:
                continue

In [19]:
# url = 'https://www.liputan6.com'
# pencarian = ''
# tanggal = '125-12-2021'
# testing = Liputan6(url, pencarian,tanggal)
# testing.getUrlDetail(4)